In [45]:
import pandas as pd

# Load the dataset
df = pd.read_csv('Alphabets_data.csv')

# Explore basic information about the dataset
print(df.info())
print(df.describe())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2ybar  20000 non-null  int64 
 12  xy2bar  20000 non-null  int64 
 13  xedge   20000 non-null  int64 
 14  xedgey  20000 non-null  int64 
 15  yedge   20000 non-null  int64 
 16  yedgex  20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB
None
               xbox          ybox         width       height         onpix  \
count  20000.000000  20000.000000  20000.000000  20000.00000  20000.00000

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Splitting features and target
X = df.drop('letter', axis=1)  # assuming 'target' is the column with class labels
y = df['letter']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the training target variable
y_train_encoded = label_encoder.fit_transform(y_train)

# Optionally, apply the transformation to the test set as well
y_test_encoded = label_encoder.transform(y_test)



In [48]:
from tensorflow.keras.models import Sequential

In [49]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [50]:
model = Sequential()

In [51]:
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))

c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [52]:
model.add(Dense(26, activation='softmax'))

In [53]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [54]:
print(X_train_scaled.shape[0], y_train.shape[0])  # Should be equal
print(X_test_scaled.shape[0], y_test.shape[0])    # Should be equal


16000 16000
4000 4000


In [58]:
print(X_train_scaled.shape)  # Should be (num_samples, num_features)
print(y_train_encoded.shape)         # Should be (num_samples,) for regression or (num_samples, num_classes) for classification


(16000, 16)
(16000,)


In [59]:
print(type(X_train_scaled)) 


print(type(y_train_encoded))         # Should be <class 'numpy.ndarray'> or a Tensor


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [60]:
history = model.fit(X_train_scaled, y_train_encoded, epochs=20, batch_size=32, validation_data=(X_test_scaled, y_test_encoded))

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2155 - loss: 3.0223 - val_accuracy: 0.4613 - val_loss: 2.1668
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5122 - loss: 1.9941 - val_accuracy: 0.5922 - val_loss: 1.6282
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6111 - loss: 1.5552 - val_accuracy: 0.6352 - val_loss: 1.4006
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6471 - loss: 1.3594 - val_accuracy: 0.6773 - val_loss: 1.2603
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6789 - loss: 1.2285 - val_accuracy: 0.7038 - val_loss: 1.1774
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6955 - loss: 1.1631 - val_accuracy: 0.7130 - val_loss: 1.1147
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7164 - loss: 1.0868 - val_accuracy: 0.7360 - val_loss: 1.0571
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7235 - loss: 1.0474 - val_accuracy: 0.

In [63]:
import keras_tuner as kt

# Define the model building function for hyperparameter tuning
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units', min_value=32, max_value=256, step=32), activation='relu', input_dim=X_train_scaled.shape[1]))
    model.add(Dense(26, activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=hp.Float('lr', min_value=1e-5, max_value=1e-2, sampling='log')), 
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, executions_per_trial=1, directory='output')

# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train_encoded, epochs=20, validation_data=(X_test_scaled, y_test_encoded))


Trial 5 Complete [00h 00m 15s]
val_accuracy: 0.6585000157356262

Best val_accuracy So Far: 0.812749981880188
Total elapsed time: 00h 01m 26s


In [65]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Evaluate the default model
y_pred = model.predict(X_test_scaled)
y_pred_classes = y_pred.argmax(axis=1)
print('Accuracy:', accuracy_score(y_test_encoded, y_pred_classes))
print('Classification Report:\n', classification_report(y_test_encoded, y_pred_classes))
print('Confusion Matrix:\n', confusion_matrix(y_test_encoded, y_pred_classes))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step
Accuracy: 0.7875
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.89      0.89       149
           1       0.61      0.84      0.71       153
           2       0.70      0.72      0.71       137
           3       0.63      0.86      0.73       156
           4       0.73      0.91      0.81       141
           5       0.74      0.82      0.78       140
           6       0.73      0.46      0.57       160
           7       0.69      0.58      0.63       144
           8       0.91      0.80      0.85       146
           9       0.81      0.86      0.83       149
          10       0.67      0.72      0.69       130
          11       0.87      0.83      0.85       155
          12       0.90      0.92      0.91       168
          13       0.91      0.82      0.86       151
          14       0.72      0.81      0.76       145
          15       0.94      0.77      0.85       1

In [66]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │         1,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,778 (10.85 KB)

 Trainable params: 2,778 (10.85 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
from sklearn.metrics import classification_report

# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels

# Generate classification report
print(classification_report(y_test_encoded, y_pred_classes))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       149
           1       0.00      0.00      0.00       153
           2       0.09      0.98      0.17       137
           3       0.00      0.00      0.00       156
           4       0.11      0.91      0.19       141
           5       0.43      0.62      0.51       140
           6       0.00      0.00      0.00       160
           7       0.43      0.26      0.32       144
           8       0.00      0.00      0.00       146
           9       0.82      0.59      0.69       149
          10       0.00      0.00      0.00       130
          11       0.00      0.00      0.00       155
          12       0.95      0.84      0.89       168
          13       1.00      0.01      0.01       151
          14       0.24      0.04      0.07       145
          15       0.99      0.42      0.59       173
          16       0.00      0.00     

c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

In [68]:
# Evaluate default model
default_model = model
y_pred_default = default_model.predict(X_test)
y_pred_classes_default = y_pred_default.argmax(axis=1)
print("Default Model Performance:\n", classification_report(y_test_encoded, y_pred_classes_default))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step
Default Model Performance:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       149
           1       0.34      0.07      0.11       153
           2       0.12      0.98      0.21       137
           3       0.97      0.22      0.36       156
           4       0.14      0.94      0.24       141
           5       0.47      0.63      0.54       140
           6       0.00      0.00      0.00       160
           7       0.39      0.28      0.33       144
           8       1.00      0.11      0.20       146
           9       0.87      0.64      0.74       149
          10       0.12      0.02      0.03       130
          11       0.00      0.00      0.00       155
          12       0.95      0.84      0.89       168
          13       1.00      0.02      0.04       151
          14       0.47      0.13      0.21       145
          15       1.00      0.34      0.50       173
          

c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\HP\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m

Key Insights:

Low Precision and Recall for Many Classes:

Precision, recall, and f1-scores are low for many of the classes (especially class 0, 6, 11, 13, and 18).

For instance, for class 0, the precision and recall are 0.00, which indicates that the model is not predicting this class at all, and even when it does, the predictions are incorrect.

For class 2, the recall is high (0.98), meaning the model is good at identifying this class, but its precision is still low (0.12), suggesting many false positives.

Accuracy:

The overall accuracy is 37%, which indicates that the model is not doing well in correctly classifying the data across all classes.
The accuracy alone isn't enough to determine the model's performance, especially in a multiclass classification problem, where a more detailed analysis using precision, recall, and f1-scores is necessary.

Imbalanced Classes:

The classes have varying numbers of instances (support column), which suggests that the dataset might be imbalanced. Some classes have significantly more samples (e.g., class 19 has 163 instances) while others have fewer (e.g., class 0 has 149 instances).

This class imbalance can lead to poor model performance, especially when the model is biased towards predicting the more frequent classes.